[INCOMPLETE] 
In this notebook we will be using computer vision to offer a forest fire identification feature. This notebook will use regular pictures of the sky (the other will attempt to utilize satellite data.) Import the necessary packages first.

In [ ]:
import os
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

Then, we locate our data for training the model and create an imagegen to preprocess it. Before we begin training our model we need to make sure our image files are all valid (not corrupted).

In [28]:
train_dir = "DATASETS/arch2DS/train"
test_dir = "DATASETS/arch2DS/test"
val_dir = "DATASETS/arch2DS/valid"

data_paths = [train_dir, test_dir, val_dir]
file_paths = []
for dataset_path in data_paths:
    for folder in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder)
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            try:
                with Image.open(file_path) as img:
                    img.verify()
            except (IOError, SyntaxError, OSError) as e:
                print(f"Removing corrupted image: {file_path}")
                file_paths.append(file_path)
print(len(file_paths))

train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')
val_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary')
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary')

0
Found 28480 images belonging to 2 classes.
Found 6300 images belonging to 2 classes.


We will be using the ResNet50 model from Tensorflow. It will take some time for it to load.

In [29]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
spare_model = base_model

We can customize the model by adding our own layers.

In [ ]:
for layer in spare_model.layers:
    layer.trainable = False

#layers
x = spare_model.output
x = GlobalAveragePooling2D()(x) #reduces spatial dimensions
x = Dense(128, activation='relu')(x) #the dense layer reduces the num featuress
x = Dense(1, activation='sigmoid')(x) # final sigmoid activation function layer for binary classification [fire/no fire]

model = Model(inputs=spare_model.input, outputs=x)
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
model.summary

<bound method Model.summary of <Functional name=functional_4, built=True>>

We can now fit the model to our data. The accuracy will slowly improve and the loss should slowly subside. For demonstrational purposes, I will target an accuracy of at least 80+. The satellite images are not very clear to begin with.

In [37]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True


In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size // 2,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size // 2,
    epochs=20
)

Epoch 1/20
445/445 ━━━━━━━━━━━━━━━━━━━━ 728s 2s/step - accuracy: 0.8576 - loss: 0.3582 - val_accuracy: 0.8696 - val_loss: 0.3343
Epoch 2/20
445/445 ━━━━━━━━━━━━━━━━━━━━ 594s 1s/step - accuracy: 0.8621 - loss: 0.3433 - val_accuracy: 0.8734 - val_loss: 0.3227
Epoch 3/20
445/445 ━━━━━━━━━━━━━━━━━━━━ 566s 1s/step - accuracy: 0.8719 - loss: 0.3208 - val_accuracy: 0.8565 - val_loss: 0.3520
Epoch 4/20
376/445 ━━━━━━━━━━━━━━━━━━━━ 3:17 3s/step - accuracy: 0.8666 - loss: 0.3330

KeyboardInterrupt: 

Now that we have our model we can load our weights to a file. It is best to load whatever progress you have than to lose it.

In [41]:
model.save('secondmod.h5')
print("Model has been saved to your directory.")

Model has been saved to your directory.


Let's check the performance of our model on some test data.

In [ ]:
val_loss, val_accuracy = model.evaluate(val_generator)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

In [ ]:
#select an image path and see the ability for it to detect a fire
img_path = "path_to_image.jpg"
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

prediction = model.predict(img_array)
if prediction[0][0] > 0.5:
    print("Fire detected!")
else:
    print("No fire detected.")

Now we have completed training our model and can use it for our app!